In [115]:
import os

In [116]:
%pwd

'c:\\Users\\2021ICTS28\\Desktop\\Chest-Cancer-Classification-using-MLflow-DVC'

In [117]:
os.chdir(r"c:\Users\2021ICTS28\Desktop\Chest-Cancer-Classification-using-MLflow-DVC")

In [118]:
%pwd

'c:\\Users\\2021ICTS28\\Desktop\\Chest-Cancer-Classification-using-MLflow-DVC'

In [119]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [120]:
from chestCNN.constants import *
from chestCNN.utils.common import read_yaml, create_directories
import tensorflow as tf

In [125]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [122]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [123]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [124]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-12-29 18:35:54,611: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-29 18:35:54,626: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-29 18:35:54,627: INFO: common: created directory at: artifacts]
[2025-12-29 18:35:54,627: INFO: common: created directory at: artifacts\training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/15
17/17 [==============================] - 13s 755ms/step - loss: 13.7719 - accuracy: 0.5405 - val_loss: 17.1390 - val_accuracy: 0.3906
Epoch 2/15
17/17 [==============================] - 14s 793ms/step - loss: 13.3806 - accuracy: 0.5637 - val_loss: 0.7253 - val_accuracy: 0.8750
Epoch 3/15
17/17 [==============================] - 13s 785ms/step - loss: 10.8255 - accuracy: 0.5907 - val_loss: 0.0681 - val_accuracy: 1.0000
Epoch 4/15
17/17 [==============================] - 14s 812ms/step - loss: 13.0645 - accuracy: 0.5560 - val_loss: 0.4139 - val_accuracy: 0.8750
Epoch 5/15
1